## Yolo V3 with Tensorflow

In [1]:
from net.yolo_top import yolov3
from data.data_pipeline import data_pipeline
from net.config import cfg
import numpy as np
import time
import tensorflow as tf
import os
% load_ext autoreload
% autoreload 2

In [2]:
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

In [3]:
def tower_loss(scope, imgs, true_boxes, istraining):
    """Calculate the total loss on a single tower running the CIFAR model.
    Args:
        scope: unique prefix string identifying the CIFAR tower, e.g. 'tower_0'
        images: Images. 4D tensor of shape [batch_size, height, width, 3].
        labels: Labels. 1D tensor of shape [batch_size].
    Returns:
        Tensor of shape [] containing the total loss for a batch of data
    """

    # Build inference Graph.
    # Build the portion of the Graph calculating the losses. Note that we will
    # assemble the total_loss using a custom function below.
    model = yolov3(imgs, true_boxes, istraining)
    _, _ = model.compute_loss()

    # Assemble all of the losses for the current tower only.
    losses = tf.get_collection('losses', scope)
        
    # Calculate the total loss for the current tower.
    total_loss = tf.add_n(losses, name='total_loss')
    
    # for multigpu test
    test_out = tf.get_collection('feats52', scope)
    total_feats52 = tf.add_n(test_out, name='total_feats52')
    
#    print("loss")
#    print(total_loss)
    
    return total_loss, total_feats52

def average_gradients(tower_grads):
    """Calculate the average gradient for each shared variable across all towers.
    Note that this function provides a synchronization point across all towers.
    Args:
        tower_grads: List of lists of (gradient, variable) tuples. The outer list
        is over individual gradients. The inner list is over the gradient
        calculation for each tower.
    Returns:
        List of pairs of (gradient, variable) where the gradient has been averaged
        across all towers.
    """
    average_grads = []

    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        
#        print(type(grad_and_vars))
#        print(grad_and_vars)
        for g, _ in grad_and_vars:
#            print("###")
#            print(type(g))
#            print(g)
            
            
            
            
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
def train(num_gpus):

    with tf.Graph().as_default(), tf.device('/cpu:0'):
        # Create a variable to count the number of train() calls. This equals the
        # number of batches processed * FLAGS.num_gpus.
        global_step = tf.Variable(0, trainable=False)
        
        decay_steps = int(100 * 100)

        #lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE, 
        #                                 global_step,
        #                                 decay_steps,
        #                                 LEARNING_RATE_DECAY_FACTOR,
        #                                 staircase=True)

        lr = tf.train.piecewise_constant(global_step, 
                                         cfg.train.lr_steps,
                                         cfg.train.lr_scales)
        
        # Create an optimizer that performs gradient descent.
        
#        opt = tf.train.GradientDescentOptimizer(lr)
        
        opt = tf.train.AdamOptimizer(learning_rate = lr)
        
        tower_grads = []
        
        test_feats = []
        
        imgs, true_boxes = data_pipeline(
            [cfg.path.train_data_path, cfg.path.train_data_path], cfg.batch_size)
        
        print(imgs.shape)
        print(true_boxes.shape)
        
        imgs = tf.reshape(imgs, 
                          [cfg.batch_size, imgs.shape[1], imgs.shape[2], imgs.shape[3]])
        true_boxes = tf.reshape(true_boxes, 
                                [cfg.batch_size, true_boxes.shape[1], true_boxes.shape[2]])
        
        batch_queue = tf.contrib.slim.prefetch_queue.prefetch_queue(
            [imgs, true_boxes], capacity=2 * num_gpus)
        
        imgs_batch = [[], []]
        true_boxes_batch = [[], []]
        
        imgs = [[], []]
        targets = [[], []]
        
        with tf.variable_scope(tf.get_variable_scope()):
            for i in range(num_gpus):
                with tf.device('/gpu:%d' % i):
                    with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                        istraining = tf.constant(True, tf.bool)
                        print(scope)
                        
                        imgs_batch[i], true_boxes_batch[i] = batch_queue.dequeue()
                        # Calculate the loss for one tower of the CIFAR model. This function
                        # constructs the entire CIFAR model but shares the variables across
                        # all towers.
                        
                        loss, test_feat = tower_loss(scope, 
                                          imgs_batch[i], 
                                          true_boxes_batch[i],
                                          istraining)

                        # Reuse variables for the next tower.
                        tf.get_variable_scope().reuse_variables()

                        # Retain the summaries from the final tower.
                        summaries = tf.get_collection(tf.GraphKeys.SUMMARIES, scope)

                        # Calculate the gradients for the batch of data on this CIFAR tower.
                        grads = opt.compute_gradients(loss)
                        
                        print('ops________________________________________________')
                        for i in grads:
                            print(i[0])
                        print ('variables________________________________________________')
                        for i in grads:
                            print (i[1])

                        # Keep track of the gradients across all towers.
                        tower_grads.append(grads)
                        
                        test_feats.append(test_feat)

        # We must calculate the mean of each gradient. Note that this is the
        # synchronization point across all towers.

        grads = average_gradients(tower_grads)
        
        test_feats = tf.reduce_mean(test_feats, 0)
        
        print('average ops________________________________________________')
        for i in grads:
            print(i[0])
        print ('average variables________________________________________________')
        for i in grads:
            print (i[1])

        # Add a summary to track the learning rate.
        summaries.append(tf.summary.scalar('learning_rate', lr))

        # Add histograms for gradients.
        for grad, var in grads:
            if grad is not None:
                summaries.append(tf.summary.histogram(var.op.name + '/gradients', grad))

        # Apply the gradients to adjust the shared variables.
        apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)
        
        #update_op = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        #vars_det = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Head")
        
        #with tf.control_dependencies(update_op):
        #    apply_gradient_op = opt.minimize(loss, 
        #                                     global_step = global_step, 
        #                                     var_list = vars_det)
        
        # Add histograms for trainable variables.
        for var in tf.trainable_variables():
            summaries.append(tf.summary.histogram(var.op.name, var))

        # Group all updates to into a single train op.
        train_op = apply_gradient_op

        # Create a saver.
        saver = tf.train.Saver(tf.global_variables())

        # Build the summary operation from the last tower summaries.
        summary_op = tf.summary.merge(summaries)

        # Build an initialization operation to run below.
        init = tf.global_variables_initializer()

        # Start running operations on the Graph. allow_soft_placement must be set to
        # True to build towers on GPU, as some of the ops do not have GPU
        # implementations.
        sess = tf.Session(config=tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False))
        sess.run(init)
        
        # Start the queue runners.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)

        for step in range(10000):
            
            
            all_imgs = sess.run(imgs_batch)
            all_true_boxes = sess.run(true_boxes_batch)
            
            start_time = time.time()
            _ = sess.run(test_feats)
            duration = time.time() - start_time
            
            print(duration)
            if (step % 10 == 0):
                #print(loss_value)
                pass

        for i in range(100):
            start_time = time.time()
            imgs = sess.run(imgs_batch[0])
            duration = time.time() - start_time
            print(duration)
            print(np.array(imgs).shape)

        print("Complete!!")
        coord.request_stop()
        coord.join(threads)
        sess.close()
            
            

In [5]:
train(2)

(?, 608, 608, 3)
(?, 30, 5)
tower_0/
build start
Tensor("tower_0/Feature_Extractor/conv_0/LeakyRelu/Maximum:0", shape=(4, 608, 608, 32), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/Feature_Extractor/conv_1/LeakyRelu/Maximum:0", shape=(4, 304, 304, 64), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/Feature_Extractor/conv_2/LeakyRelu/Maximum:0", shape=(4, 304, 304, 32), dtype=float32, device=/device:GPU:0)
build done
ops________________________________________________
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/batchnorm/sub_grad/tuple/control_dependency:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Feature_Extractor/conv_0/batchnorm/mul_grad/tuple/control_dependency_1:0", shape=(32,), dtype=float32, device=/device:GPU:0)
Tensor("tower_0/gradients/tower_0/Fea

build done
ops________________________________________________
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/batchnorm/sub_grad/tuple/control_dependency:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/batchnorm/mul_grad/tuple/control_dependency_1:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/cond/Switch_1_grad/cond_grad:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_0/cond/Switch_2_grad/cond_grad:0", shape=(32,), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/gradients/tower_1/Feature_Extractor/conv_1/Conv2D_grad/tuple/control_dependency_1:0", shape=(3, 3, 32, 64), dtype=float32, device=/device:GPU:1)
Tensor("tower_1/

average ops________________________________________________
Tensor("Mean:0", shape=(3, 3, 3, 32), dtype=float32, device=/device:CPU:0)
Tensor("Mean_1:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_2:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_3:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_4:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_5:0", shape=(3, 3, 32, 64), dtype=float32, device=/device:CPU:0)
Tensor("Mean_6:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_7:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_8:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_9:0", shape=(64,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_10:0", shape=(1, 1, 64, 32), dtype=float32, device=/device:CPU:0)
Tensor("Mean_11:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_12:0", shape=(32,), dtype=float32, device=/device:CPU:0)
Tensor("Mean_13:

2.4127750396728516
0.18554067611694336
0.19059348106384277
0.18262648582458496
0.20299386978149414
0.18959784507751465
0.2052595615386963
0.18860721588134766
0.1982893943786621
0.20588994026184082
0.19087743759155273
0.19833135604858398
0.19031548500061035
0.19157814979553223
0.19490504264831543
0.21110033988952637
0.19052362442016602
0.21386289596557617
0.2086331844329834
0.2002854347229004
0.19402194023132324
0.19890761375427246
0.18446707725524902
0.18780517578125
0.19161391258239746
0.19760966300964355
0.19629621505737305
0.1966705322265625
0.19560694694519043
0.18358993530273438
0.21491217613220215
0.19966912269592285
0.1897599697113037
0.19210219383239746
0.19172167778015137
0.20176291465759277
0.2001042366027832
0.20902037620544434
0.1911931037902832
0.20329594612121582
0.19028043746948242
0.19248604774475098
0.19044804573059082
0.17931866645812988
0.2104473114013672
0.1942744255065918
0.21199250221252441
0.19107317924499512
0.20087289810180664
0.1928102970123291
0.1921598911285

0.21069717407226562
0.18938708305358887
0.19302725791931152
0.20641589164733887
0.19147396087646484
0.21806073188781738
0.1857452392578125
0.1979992389678955
0.20641422271728516
0.18252062797546387
0.19579124450683594
0.22728919982910156
0.1916654109954834
0.19785499572753906
0.2007157802581787
0.21585917472839355
0.21791291236877441
0.1871042251586914
0.19465255737304688
0.20026469230651855
0.1928551197052002
0.20499515533447266
0.19324636459350586
0.20299315452575684
0.22214245796203613
0.19068098068237305
0.20033884048461914
0.19452357292175293
0.18421602249145508
0.21726202964782715
0.18912959098815918
0.1893754005432129
0.20762157440185547
0.20029425621032715
0.21459531784057617
0.19103288650512695
0.19881629943847656
0.18468832969665527
0.1795811653137207
0.20100808143615723
0.21370768547058105
0.18647480010986328
0.20212244987487793
0.18825674057006836
0.1880476474761963
0.19948482513427734
0.18455004692077637
0.1915726661682129
0.17437386512756348
0.20172858238220215
0.20204663

...

0.20983457565307617
0.1941986083984375
0.19001317024230957
0.20244479179382324
0.19083571434020996
0.20443964004516602
0.1957077980041504
0.18178367614746094
0.1998453140258789
0.19352173805236816
0.2010951042175293
0.19864320755004883
0.19611644744873047
0.19397616386413574
0.22005534172058105
0.1964397430419922
0.19066691398620605
0.1727752685546875
0.2039186954498291
0.1961512565612793
0.19182467460632324
0.19984054565429688
0.2221841812133789
0.19194936752319336
0.21409916877746582
0.19337916374206543
0.19390344619750977
0.1817626953125
0.18988752365112305
0.19036078453063965
0.18368172645568848
0.2024211883544922
0.23084807395935059
0.19504976272583008
0.2038569450378418
0.19833040237426758
0.1948084831237793
0.18198490142822266
0.19982552528381348
0.20734167098999023
0.1927964687347412
0.20716094970703125
0.19385242462158203
0.21162080764770508
0.18753790855407715
0.19612598419189453
0.18236827850341797
0.17790818214416504
0.19407129287719727
0.18602395057678223
0.192139625549316

0.3739628791809082
0.18398737907409668
0.2056732177734375
0.18821287155151367
0.19707465171813965
0.19725656509399414
0.18435049057006836
0.20317983627319336
0.18827557563781738
0.19250130653381348
0.21131396293640137
0.19175028800964355
0.2044355869293213
0.19829988479614258
0.19376349449157715
0.19360947608947754
0.18375587463378906
0.20157504081726074
0.18888068199157715
0.2054898738861084
0.19811463356018066
0.1852741241455078
0.19493842124938965
0.19655919075012207
0.22355937957763672
0.1770763397216797
0.18215203285217285
0.1862499713897705
0.19042134284973145
0.19804787635803223
0.18905091285705566
0.19611024856567383
0.19956493377685547
0.19014811515808105
0.201185941696167
0.2056865692138672
0.18371152877807617
0.1940460205078125
0.20052623748779297
0.19272685050964355
0.1999669075012207
0.19414687156677246
0.1972041130065918
0.19094133377075195
0.19522619247436523
0.20364046096801758
0.17905473709106445
0.1965954303741455
0.20166397094726562
0.21007323265075684
0.190965175628

0.19962692260742188
0.20242929458618164
0.1930227279663086
0.2030344009399414
0.17824888229370117
0.21516942977905273
0.23434948921203613
0.19695091247558594
0.20212244987487793
0.19167876243591309
0.2028951644897461
0.19925141334533691
0.19176244735717773
0.1785264015197754
0.19400525093078613
0.20274090766906738
0.20237970352172852
0.22899842262268066
0.19368958473205566
0.1990041732788086
0.19468116760253906
0.20703601837158203
0.1918959617614746
0.18895196914672852
0.18751907348632812
0.20181894302368164
0.1942765712738037
0.20228171348571777
0.1907968521118164
0.2211925983428955
0.18828368186950684
0.1950376033782959
0.19186758995056152
0.17795848846435547
0.19906330108642578
0.19013547897338867
0.20746731758117676
0.2013564109802246
0.2094264030456543
0.19735479354858398
0.1990211009979248
0.19356536865234375
0.21230220794677734
0.1800377368927002
0.19715619087219238
0.19515275955200195
0.1989305019378662
0.19362545013427734
0.18771028518676758
0.18910956382751465
0.1960332393646

0.21333813667297363
0.20479893684387207
0.20081806182861328
0.17315220832824707
0.19000530242919922
0.19308137893676758
0.19416189193725586
0.20311832427978516
0.20763945579528809
0.20734882354736328
0.1876227855682373
0.2013695240020752
0.22729277610778809
0.17382287979125977
0.19039368629455566
0.20874834060668945
0.19216656684875488
0.18827223777770996
0.19499731063842773
0.20229434967041016
0.18974733352661133
0.19838500022888184
0.19325852394104004
0.17830395698547363
0.18517136573791504
0.18523192405700684
0.20339512825012207
0.20094633102416992
0.2038710117340088
0.19683361053466797
0.18935394287109375
0.20372438430786133
0.19266819953918457
0.1827106475830078
0.20537590980529785
0.1882648468017578
0.2046511173248291
0.21413087844848633
0.20032215118408203
0.21912527084350586
0.20473766326904297
0.19765186309814453
0.19309449195861816
0.19698643684387207
0.18986725807189941
0.192976713180542
0.1947612762451172
0.19462847709655762
0.1882312297821045
0.207794189453125
0.1948301792

KeyboardInterrupt: 